In [1]:
using LinearAlgebra
using Distributions, LinearAlgebra
using DifferentialEquations

In [2]:
include("../src/models.jl")
include("../src/time_series.jl")
include("../src/state_space.jl")
include("../src/catalog.jl")
include("../src/generate_data.jl")

generate_data

In [3]:
?StateSpaceModel

search: StateSpaceModel



Space-State model is defined through the following equations

$$
\left\{
\begin{array}{l}
X_t = m(X_{t-1}) + \eta_t, \\
Y_t = H(X_t) + \varepsilon_t,
\end{array}
\right.
$$

  * X : hidden variables
  * Y : observed variables
  * `dt_integration`is the numerical time step used to solve the ODE.
  * `dt_states` is the number of `dt_integration` between $X_{t-1}$ and $X_t$.
  * `dt_obs` is the number of `dt_integration` between $Y_{t-1}$ and $Y_t$.


In [4]:
σ = 10.0
ρ = 28.0
β = 8.0/3

dt_integration = 0.01
dt_states      = 1 
dt_obs         = 8 
parameters     = [σ, ρ, β]
var_obs        = [1]
nb_loop_train  = 30 
nb_loop_test   = 1
sigma2_catalog = 0.0
sigma2_obs     = 2.0

ssm = StateSpaceModel( dt_integration, dt_states, dt_obs, 
                       parameters, var_obs,
                       nb_loop_train, nb_loop_test,
                       sigma2_catalog, sigma2_obs )

xt, yo, catalog = generate_data( ssm )

(TimeSeries{Float64}(101, 3, [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], [7.25668 11.744 16.9675; 7.71695 12.4326 17.4143; … ; 12.4919 5.0265 38.8452; 11.7159 3.69806 38.3431]), TimeSeries{Float64}(101, 3, [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], [4.87858 NaN NaN; NaN NaN NaN; … ; NaN NaN NaN; NaN NaN NaN]), Catalog{Float64}([11.7159 3.69806 38.3431; 11.7079 3.68558 38.3372; … ; -2.5464 -4.51909 21.222; -2.54837 -4.52037 21.2175], [11.7159 3.69806 38.3431; 11.7079 3.68558 38.3372; … ; -2.54443 -4.51782 21.2265; -2.5464 -4.51909 21.222], [11.7079 3.68558 38.3372; 11.6999 3.67311 38.3313; … ; -2.5464 -4.51909 21.222; -2.54837 -4.52037 21.2175], StateSpaceModel(0.01, 1, 8, [10.0, 28.0, 2.66667], [1], 30, 1, 0.0, 2.0)))

In [5]:
include("../src/model_forecasting.jl")

mf = ModelForecasting( ssm )

ModelForecasting(0.01, [10.0, 28.0, 2.66667])

In [6]:
μ = [0.,0.,0]
σ = 0.1 .* Matrix(I,3,3)
d = MvNormal(μ, σ)
np = 10
x = [8.0 0.0 30.0] .+ rand(d, np)'
mf(x)

10×3 Array{Float64,2}:
 7.02601   0.277035   29.9468
 7.55585   0.0838744  29.2642
 7.07883  -0.90367    29.2677
 6.84148   0.555643   29.7196
 7.02075  -0.328578   28.9095
 7.16764  -0.338593   29.3334
 7.07062  -0.348486   28.6547
 7.18853   0.228811   29.1208
 7.03164  -0.136325   28.9508
 7.27569  -0.218861   29.2762

In [7]:
xb = xt.values[1,:]
xf = xb' .+ rand(d, np)'


10×3 Array{Float64,2}:
 7.39926  11.8469  16.7588
 7.38928  11.2219  17.2274
 7.43061  11.409   16.9929
 7.66755  11.5564  16.983 
 6.78382  11.4279  16.5447
 7.05006  11.2902  16.9534
 7.33631  11.7271  16.9041
 7.75062  11.6664  16.9366
 7.35586  11.8377  17.7479
 7.12964  11.6919  17.4737

In [26]:
include("../src/data_assimilation.jl")
da = DataAssimilation( mf, :EnKs, np, xt, ssm.sigma2_obs)
data_assimilation(yo, da)

1-element Array{Int64,1}:
 1

-

In [27]:
findall(.!isnan.(yo.values[k,1]))


UndefVarError: UndefVarError: k not defined